In [ ]:
%%time
import pandas as pd
import numpy
from collections import Counter
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('sportsv_baseball_categroy.csv', sep='|')

# ckiplab word segment (中研院斷詞)
# Initialize drivers
# It takes time to download ckiplab models

# default參數是model="bert-base"
# ws = CkipWordSegmenter() 
# pos = CkipPosTagger()
# ner = CkipNerChunker()

# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny") 
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")


## Word Segmentation
tokens = ws(df.content)

## POS
tokens_pos = pos(tokens)

## word pos pair 詞性關鍵字
word_pos_pair = [list(zip(w, p)) for w, p in zip(tokens, tokens_pos)]

## NER命名實體辨識
entity_list = ner(df.content)

# Remove stop words and filter using POS tag (tokens_v2)
#with open('stops_chinese_traditional.txt', 'r', encoding='utf8') as f:
#    stops = f.read().split('\n')

# 過濾條件:兩個字以上 特定的詞性
# allowPOS 過濾條件: 特定的詞性
allowPOS = ['Na', 'Nb', 'Nc', 'VC']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

# Insert tokens into dataframe (新增斷詞資料欄位)
df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

# Calculate word count (frequency) 計算字頻(次數)


def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

# Abstract (summary) and sentimental score(摘要與情緒分數)
summary = []
sentiment = []
for text in df.content:
    summary.append("暫無")
    sentiment.append("暫無")

df['summary'] = summary
df['sentiment'] = sentiment

# Rearrange the colmun order for readability
df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

# Save data to disk
df.to_csv('sportsv_preprocessed.csv', sep='|', index=False)

## Read it out 讀出看看
#df = pd.read_csv('cna_dataset_preprocessed.csv', sep='|')
#df.head(1)

print("Tokenize OK!")

c:\Users\user\anaconda3\envs\ai24\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\ai24\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--ckiplab--albert-tiny-chinese-ws. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\ai24\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--ckiplab--albert-tiny-chinese-ner. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/16.0M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
## Read it out 讀出看看
df = pd.read_csv('sportsv_preprocessed.csv', sep='|')
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,sportsv_20250319_1,2025-03-19,MLB,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,暫無,暫無,"[('道奇', 4), ('威力', 4), ('東京', 3), ('佐佐木朗希', 3)...","['《', '看看', 'MLB25-05', '》', '國聯', '雙雄', '的', ...","['國聯', '雙雄', '東京', '系列賽', '回合', '佐佐木朗希', '大聯盟'...","[NerToken(word='第二', ner='ORDINAL', idx=(22, 2...","[('《', 'PARENTHESISCATEGORY'), ('看看', 'VE'), (...",https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...


In [4]:
import pandas as pd
import numpy

In [5]:
df = pd.read_csv('sportsv_baseball_categroy.csv', sep='|')

In [6]:
df.shape

(290, 7)

In [7]:
df.head()

,item_id,date,title,category,content,link,photo_link
0,sportsv_20250319_1,2025-03-19,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,MLB,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...
1,sportsv_20250319_2,2025-03-19,芝加哥小熊vs洛杉磯道奇 文字直播：大谷翔平本季首轟在內單場三轟，道奇隊兩戰橫掃小熊隊,MLB,2025 大聯盟賽季在昨日（3/19）於東京巨蛋正式拉開帷幕，這不僅是大聯盟歷史上首次的日本...,https://www.sportsv.net/articles/118233,https://bucket-image.inkmaginecms.com/version/...
2,sportsv_20210429_3,2021-04-29,棒球迷的最佳選擇：最新《MLB九局職棒21》，帶給玩家更愛不釋手的遊戲體驗,MLB,約莫在2005年，王建民在大聯盟的表現開始加溫，從2005年的8勝到2006年、2007年的...,https://www.sportsv.net/articles/83288,https://bucket-image.inkmaginecms.com/version/...
3,sportsv_20250319_4,2025-03-19,「未達到指定局數就取消指定打擊」，Max Scherzer 提出當今先發投手局數過短的解決方案,MLB,"此文編譯自To Save Starting Pitching, Max Scherzer T...",https://www.sportsv.net/articles/118203,https://bucket-image.inkmaginecms.com/version/...
4,sportsv_20250319_5,2025-03-19,與山本由伸聯手貢獻關鍵戰力 大谷翔平坦言難得感到緊張,MLB,美國職棒大聯盟（MLB）新球季揭幕戰登場，3月18日晚間在東京巨蛋由洛杉磯道奇對決芝加哥小熊...,https://www.sportsv.net/articles/118229,https://bucket-image.inkmaginecms.com/version/...


In [8]:
df.content[0:5]

0    《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...
1    2025 大聯盟賽季在昨日（3/19）於東京巨蛋正式拉開帷幕，這不僅是大聯盟歷史上首次的日本...
2    約莫在2005年，王建民在大聯盟的表現開始加溫，從2005年的8勝到2006年、2007年的...
3    此文編譯自To Save Starting Pitching, Max Scherzer T...
4    美國職棒大聯盟（MLB）新球季揭幕戰登場，3月18日晚間在東京巨蛋由洛杉磯道奇對決芝加哥小熊...
Name: content, dtype: object

In [9]:
df['content'][0:5]

0    《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...
1    2025 大聯盟賽季在昨日（3/19）於東京巨蛋正式拉開帷幕，這不僅是大聯盟歷史上首次的日本...
2    約莫在2005年，王建民在大聯盟的表現開始加溫，從2005年的8勝到2006年、2007年的...
3    此文編譯自To Save Starting Pitching, Max Scherzer T...
4    美國職棒大聯盟（MLB）新球季揭幕戰登場，3月18日晚間在東京巨蛋由洛杉磯道奇對決芝加哥小熊...
Name: content, dtype: object

In [10]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

In [11]:
# model="albert-tiny" 模型小，斷詞速度比較快，犧牲一些精確度
ws = CkipWordSegmenter(model="albert-tiny")
pos = CkipPosTagger(model="albert-tiny")
ner = CkipNerChunker(model="albert-tiny")

RuntimeError: Failed to import transformers.models.albert.modeling_albert because of the following error (look up to see its traceback):
Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
%%time
# 確保 'content' 欄位為字串，並填補 NaN 避免錯誤
df['content'] = df['content'].astype(str).fillna('')

# 進行斷詞（轉為 list 確保可迭代）
tokens = ws(df['content'].tolist())

Inference: 100%|██████████| 1/1 [00:03<00:00,  3.42s/it]

CPU times: total: 25.7 s
Wall time: 3.6 s


In [ ]:
len(tokens)

36

In [ ]:
len(tokens[0])

922

In [ ]:
tokens[0]

['《',
 '看看',
 'MLB25-05',
 '》',
 '國聯',
 '雙雄',
 '的',
 '東京',
 '系列賽',
 '第二',
 '回合',
 '，',
 '除了',
 '小',
 '熊',
 '企圖',
 '想',
 '扳回',
 '一',
 '城',
 '外',
 '，',
 '更',
 '是',
 '佐佐木朗希',
 '大聯盟',
 '的',
 '初',
 '登板',
 '；',
 '開',
 '場',
 '雖',
 '百英里',
 '速球',
 '連發',
 '，',
 '卻',
 '在',
 '控球',
 '方面',
 '遇到',
 '麻煩',
 '，',
 '最終',
 '繳',
 '出',
 '3',
 '局',
 '1',
 '安1',
 '失分',
 '5',
 '保送',
 '3',
 '三',
 '振',
 '成績單',
 '。',
 '道奇',
 '打線',
 '即便',
 '串',
 '連頻',
 '遭',
 '對手',
 '阻斷',
 '，',
 '但',
 '全壘打',
 '炸',
 '裂',
 '的',
 '不',
 '手軟',
 '，',
 '連同',
 '大谷翔平',
 '首',
 '轟',
 '在',
 '內',
 '用',
 '3',
 '發',
 '攻下',
 '6',
 '分',
 '，',
 '縱使',
 '小',
 '熊',
 '攻防',
 '狀態',
 '逐漸',
 '回溫',
 '，',
 '也',
 '無力',
 '回天',
 '並',
 '吞下',
 '二',
 '連敗',
 '。',
 '延伸',
 '閱讀',
 '：',
 '芝加哥',
 '小熊',
 'vs',
 '洛杉磯',
 '道奇',
 ' ',
 '文字',
 '直',
 '播',
 '：',
 '大谷翔平',
 '本季',
 '首轟在',
 '內',
 '單場',
 '三轟',
 '，',
 '道奇隊',
 '兩戰',
 '橫掃',
 '小',
 '熊隊',
 'M',
 'LB',
 '東京',
 '系列賽',
 '第二',
 '回合',
 '，',
 '不僅',
 '是',
 '小熊',
 '想',
 '扳回',
 '一',
 '城',
 '，',
 '更',
 '是',
 '休賽季',
 '引發',
 '喧然',
 '

In [ ]:
tokens[0][0]

'《'

In [ ]:
%%time
tokens_pos = pos(tokens)

Inference: 100%|██████████| 14/14 [00:18<00:00,  1.33s/it]

CPU times: total: 2min 36s
Wall time: 19.2 s


In [ ]:
len(tokens_pos)

36

In [ ]:
len(tokens_pos[0])

922

In [ ]:
tokens_pos[0]

['PARENTHESISCATEGORY',
 'VE',
 'FW',
 'PARENTHESISCATEGORY',
 'Nb',
 'Na',
 'DE',
 'Nc',
 'Na',
 'Neu',
 'Na',
 'COMMACATEGORY',
 'P',
 'VH',
 'Na',
 'VF',
 'VE',
 'VC',
 'Neu',
 'Na',
 'Ng',
 'COMMACATEGORY',
 'Dfa',
 'SHI',
 'Nb',
 'Na',
 'DE',
 'D',
 'VA',
 'SEMICOLONCATEGORY',
 'VA',
 'Nf',
 'Cbb',
 'Nb',
 'Na',
 'VJ',
 'COMMACATEGORY',
 'D',
 'P',
 'Nv',
 'Na',
 'VC',
 'Na',
 'COMMACATEGORY',
 'D',
 'VD',
 'VC',
 'Neu',
 'Nf',
 'Neu',
 'VC',
 'VH',
 'FW',
 'VC',
 'Neu',
 'VC',
 'VH',
 'Na',
 'PERIODCATEGORY',
 'Nb',
 'VA',
 'Cbb',
 'Na',
 'Cbb',
 'P',
 'Na',
 'VC',
 'COMMACATEGORY',
 'Cbb',
 'Na',
 'VC',
 'VH',
 'DE',
 'D',
 'VH',
 'COMMACATEGORY',
 'P',
 'Nb',
 'D',
 'VC',
 'P',
 'Ncd',
 'VC',
 'Neu',
 'Nf',
 'VC',
 'Neu',
 'Nf',
 'COMMACATEGORY',
 'Cbb',
 'VH',
 'Na',
 'VC',
 'Na',
 'D',
 'VH',
 'COMMACATEGORY',
 'D',
 'D',
 'VA',
 'Cbb',
 'VC',
 'Neu',
 'VH',
 'PERIODCATEGORY',
 'VH',
 'VC',
 'COLONCATEGORY',
 'Nc',
 'VH',
 'FW',
 'Nc',
 'Nb',
 'WHITESPACE',
 'Na',
 'D',
 'VC'

In [ ]:
word_pos_pair = [list(zip(w,p)) for w, p in zip(tokens, tokens_pos)]

In [ ]:
len(word_pos_pair)

36

In [ ]:
word_pos_pair[0][0]

('《', 'PARENTHESISCATEGORY')

In [ ]:
word_pos_pair

[[('《', 'PARENTHESISCATEGORY'),
  ('看看', 'VE'),
  ('MLB25-05', 'FW'),
  ('》', 'PARENTHESISCATEGORY'),
  ('國聯', 'Nb'),
  ('雙雄', 'Na'),
  ('的', 'DE'),
  ('東京', 'Nc'),
  ('系列賽', 'Na'),
  ('第二', 'Neu'),
  ('回合', 'Na'),
  ('，', 'COMMACATEGORY'),
  ('除了', 'P'),
  ('小', 'VH'),
  ('熊', 'Na'),
  ('企圖', 'VF'),
  ('想', 'VE'),
  ('扳回', 'VC'),
  ('一', 'Neu'),
  ('城', 'Na'),
  ('外', 'Ng'),
  ('，', 'COMMACATEGORY'),
  ('更', 'Dfa'),
  ('是', 'SHI'),
  ('佐佐木朗希', 'Nb'),
  ('大聯盟', 'Na'),
  ('的', 'DE'),
  ('初', 'D'),
  ('登板', 'VA'),
  ('；', 'SEMICOLONCATEGORY'),
  ('開', 'VA'),
  ('場', 'Nf'),
  ('雖', 'Cbb'),
  ('百英里', 'Nb'),
  ('速球', 'Na'),
  ('連發', 'VJ'),
  ('，', 'COMMACATEGORY'),
  ('卻', 'D'),
  ('在', 'P'),
  ('控球', 'Nv'),
  ('方面', 'Na'),
  ('遇到', 'VC'),
  ('麻煩', 'Na'),
  ('，', 'COMMACATEGORY'),
  ('最終', 'D'),
  ('繳', 'VD'),
  ('出', 'VC'),
  ('3', 'Neu'),
  ('局', 'Nf'),
  ('1', 'Neu'),
  ('安1', 'VC'),
  ('失分', 'VH'),
  ('5', 'FW'),
  ('保送', 'VC'),
  ('3', 'Neu'),
  ('三', 'VC'),
  ('振', 'VH'),
  ('成績單', 'N

In [ ]:
%%time
entity_list = ner(df.content)

Inference: 100%|██████████| 1/1 [00:03<00:00,  3.52s/it]

CPU times: total: 25.5 s
Wall time: 3.75 s


In [ ]:
entity_list[0]

[NerToken(word='第二', ner='ORDINAL', idx=(22, 24)),
 NerToken(word='一', ner='CARDINAL', idx=(36, 37)),
 NerToken(word='佐佐木朗希大聯盟', ner='ORG', idx=(42, 50)),
 NerToken(word='百英里', ner='ORG', idx=(58, 61)),
 NerToken(word='3', ner='CARDINAL', idx=(81, 82)),
 NerToken(word='1', ner='CARDINAL', idx=(83, 84)),
 NerToken(word='3', ner='CARDINAL', idx=(91, 92)),
 NerToken(word='三', ner='CARDINAL', idx=(92, 93)),
 NerToken(word='道奇', ner='ORG', idx=(98, 100)),
 NerToken(word='洛杉磯道奇 ', ner='PERSON', idx=(179, 185)),
 NerToken(word='道奇隊', ner='ORG', idx=(205, 208)),
 NerToken(word='東京', ner='ORG', idx=(218, 220)),
 NerToken(word='二局藉', ner='ORG', idx=(348, 351)),
 NerToken(word='二局', ner='ORG', idx=(398, 400)),
 NerToken(word='Jon Berti', ner='ORG', idx=(427, 436)),
 NerToken(word='nández', ner='ORG', idx=(510, 516)),
 NerToken(word='三', ner='CARDINAL', idx=(522, 523)),
 NerToken(word='三', ner='CARDINAL', idx=(542, 543)),
 NerToken(word='5', ner='CARDINAL', idx=(549, 550)),
 NerToken(word='6：1', n

In [ ]:
# 過濾條件:兩個字以上與特定的詞性
allowPOS=['Na','Nb','Nc']

tokens_v2 =[]
for wp in word_pos_pair:
    tokens_v2.append([w for w,p in wp if (len(w) >= 2) and p in allowPOS])

In [ ]:
tokens_v2[0]

['國聯',
 '雙雄',
 '東京',
 '系列賽',
 '回合',
 '佐佐木朗希',
 '大聯盟',
 '百英里',
 '速球',
 '方面',
 '麻煩',
 '成績單',
 '道奇',
 '對手',
 '全壘打',
 '大谷翔平',
 '狀態',
 '芝加哥',
 '洛杉磯',
 '道奇',
 '文字',
 '大谷翔平',
 '單場',
 '道奇隊',
 '熊隊',
 '東京',
 '系列賽',
 '回合',
 '大波',
 '佐佐木朗希',
 '日職',
 '火球男',
 '聯盟',
 '表現',
 '球速上',
 '令和',
 '怪物',
 '快刀斬亂麻',
 '理想',
 '隊友',
 '道奇打線',
 '援護',
 '牲打',
 '雙殺打有成',
 '量能',
 '藍衫軍',
 '大谷翔平',
 '投手群',
 '幅度',
 '安打',
 '長打',
 '主軸',
 '牛棚',
 '內野',
 '對手',
 '攻勢',
 '投手',
 '道奇',
 '威力',
 '美技',
 '上述',
 '官網',
 '愛爾達',
 '官方',
 '生涯',
 '佐佐木朗希',
 '表情看來',
 '整體',
 '表現',
 '成績單',
 '球速',
 '壓迫感',
 '企圖心滿溢',
 '壞球',
 '變化',
 '威力',
 '揮空率',
 '球率',
 '空間',
 '佐佐木',
 '上策',
 '好球',
 '策略',
 '速度',
 '困擾',
 '擊球',
 '危機',
 '佐佐木',
 '好球',
 '剛猛',
 '威力',
 '策略',
 '感覺',
 '速度',
 '前述',
 '處於',
 '指叉球',
 '威嚇力',
 '好球帶',
 '佐佐木',
 '球種',
 '威力',
 '速球',
 '節奏',
 '變化',
 '心態',
 '考驗',
 '空間',
 '問題',
 '上述',
 '圖片',
 '來源',
 '運動',
 '視界',
 '愛爾達',
 '官方',
 '機會',
 '道奇',
 '火力',
 '棒頭',
 '東京',
 '巨蛋',
 '打者天堂',
 '初速',
 '仰角',
 '雙安',
 '速球',
 '暴力',
 '中外野',
 '全壘打']

In [ ]:
df['tokens'] = tokens

In [ ]:
df['tokens_v2'] = tokens_v2

In [ ]:
df['entities'] = entity_list

In [ ]:
df['token_pos'] = word_pos_pair

In [ ]:
df.head(1)

,item_id,date,title,category,content,link,photo_link,tokens,tokens_v2,entities,token_pos
0,sportsv_20250319_1,2025-03-19,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,MLB,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...,"[《, 看看, MLB25-05, 》, 國聯, 雙雄, 的, 東京, 系列賽, 第二, 回...","[國聯, 雙雄, 東京, 系列賽, 回合, 佐佐木朗希, 大聯盟, 百英里, 速球, 方面,...","[(第二, ORDINAL, (22, 24)), (一, CARDINAL, (36, 3...","[(《, PARENTHESISCATEGORY), (看看, VE), (MLB25-05..."


In [ ]:
# allowPOS過濾條件:兩個字以上與特定的詞性
# allowPOS=['Na','Nb','Nc','VA','VAC','VB','VC']
allowPOS=['Na','Nb','Nc']

from collections import Counter
def word_frequency( wp_pair ):
    filtered_words =[]
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)
        #print('%s %s' % (word, pos))
    counter = Counter( filtered_words)
    return counter.most_common(20)

In [ ]:
word_pos_pair[0]

[('《', 'PARENTHESISCATEGORY'),
 ('看看', 'VE'),
 ('MLB25-05', 'FW'),
 ('》', 'PARENTHESISCATEGORY'),
 ('國聯', 'Nb'),
 ('雙雄', 'Na'),
 ('的', 'DE'),
 ('東京', 'Nc'),
 ('系列賽', 'Na'),
 ('第二', 'Neu'),
 ('回合', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('除了', 'P'),
 ('小', 'VH'),
 ('熊', 'Na'),
 ('企圖', 'VF'),
 ('想', 'VE'),
 ('扳回', 'VC'),
 ('一', 'Neu'),
 ('城', 'Na'),
 ('外', 'Ng'),
 ('，', 'COMMACATEGORY'),
 ('更', 'Dfa'),
 ('是', 'SHI'),
 ('佐佐木朗希', 'Nb'),
 ('大聯盟', 'Na'),
 ('的', 'DE'),
 ('初', 'D'),
 ('登板', 'VA'),
 ('；', 'SEMICOLONCATEGORY'),
 ('開', 'VA'),
 ('場', 'Nf'),
 ('雖', 'Cbb'),
 ('百英里', 'Nb'),
 ('速球', 'Na'),
 ('連發', 'VJ'),
 ('，', 'COMMACATEGORY'),
 ('卻', 'D'),
 ('在', 'P'),
 ('控球', 'Nv'),
 ('方面', 'Na'),
 ('遇到', 'VC'),
 ('麻煩', 'Na'),
 ('，', 'COMMACATEGORY'),
 ('最終', 'D'),
 ('繳', 'VD'),
 ('出', 'VC'),
 ('3', 'Neu'),
 ('局', 'Nf'),
 ('1', 'Neu'),
 ('安1', 'VC'),
 ('失分', 'VH'),
 ('5', 'FW'),
 ('保送', 'VC'),
 ('3', 'Neu'),
 ('三', 'VC'),
 ('振', 'VH'),
 ('成績單', 'Na'),
 ('。', 'PERIODCATEGORY'),
 ('道奇', 'Nb'),
 ('打線', 'VA'

In [ ]:
word_frequency(word_pos_pair[3])

[('投手', 18),
 ('任務', 11),
 ('球隊', 8),
 ('問題', 4),
 ('標準', 4),
 ('季後賽', 4),
 ('數據', 3),
 ('打者', 3),
 ('局數', 3),
 ('風險', 3),
 ('場數', 3),
 ('運動', 3),
 ('打擊', 2),
 ('大聯盟', 2),
 ('趨勢', 2),
 ('球速', 2),
 ('轉速', 2),
 ('規則', 2),
 ('長局數', 2),
 ('比例', 2)]

In [ ]:
%%time
keyfreqs =[]
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

CPU times: total: 15.6 ms
Wall time: 11 ms


In [ ]:
keyfreqs[0:1]

[[('道奇', 4),
  ('威力', 4),
  ('東京', 3),
  ('佐佐木朗希', 3),
  ('速球', 3),
  ('大谷翔平', 3),
  ('佐佐木', 3),
  ('系列賽', 2),
  ('回合', 2),
  ('成績單', 2),
  ('對手', 2),
  ('全壘打', 2),
  ('表現', 2),
  ('上述', 2),
  ('愛爾達', 2),
  ('官方', 2),
  ('變化', 2),
  ('空間', 2),
  ('好球', 2),
  ('策略', 2)]]

In [ ]:
df['top_key_freq'] = keyfreqs

In [ ]:
df.head(2)

,item_id,date,title,category,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq
0,sportsv_20250319_1,2025-03-19,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,MLB,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...,"[《, 看看, MLB25-05, 》, 國聯, 雙雄, 的, 東京, 系列賽, 第二, 回...","[國聯, 雙雄, 東京, 系列賽, 回合, 佐佐木朗希, 大聯盟, 百英里, 速球, 方面,...","[(第二, ORDINAL, (22, 24)), (一, CARDINAL, (36, 3...","[(《, PARENTHESISCATEGORY), (看看, VE), (MLB25-05...","[(道奇, 4), (威力, 4), (東京, 3), (佐佐木朗希, 3), (速球, 3..."
1,sportsv_20250319_2,2025-03-19,芝加哥小熊vs洛杉磯道奇 文字直播：大谷翔平本季首轟在內單場三轟，道奇隊兩戰橫掃小熊隊,MLB,2025 大聯盟賽季在昨日（3/19）於東京巨蛋正式拉開帷幕，這不僅是大聯盟歷史上首次的日本...,https://www.sportsv.net/articles/118233,https://bucket-image.inkmaginecms.com/version/...,"[2025 , 大聯盟, 賽季, 在, 昨日, （, 3/19, ）, 於, 東京, 巨蛋,...","[大聯盟, 東京, 巨蛋, 帷幕, 大聯盟, 歷史, 日本, 投手, 山本由, 決今, 永昇...","[(昨日, DATE, (11, 13)), (東京, ORG, (20, 22)), (日...","[(2025 , Neu), (大聯盟, Na), (賽季, Nd), (在, P), (昨...","[(日本, 4), (大聯盟, 3), (東京, 2), (巨蛋, 2), (開幕戰, 2)..."


In [ ]:
df.iloc[0].top_key_freq

[('道奇', 4),
 ('威力', 4),
 ('東京', 3),
 ('佐佐木朗希', 3),
 ('速球', 3),
 ('大谷翔平', 3),
 ('佐佐木', 3),
 ('系列賽', 2),
 ('回合', 2),
 ('成績單', 2),
 ('對手', 2),
 ('全壘打', 2),
 ('表現', 2),
 ('上述', 2),
 ('愛爾達', 2),
 ('官方', 2),
 ('變化', 2),
 ('空間', 2),
 ('好球', 2),
 ('策略', 2)]

In [ ]:
%%time
summary=[]
sentiment=[]
for text in df.content: # process piece by piece
    summary.append("暫無")
    sentiment.append("暫無")

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
df['summary'] = summary

In [ ]:
df['sentiment'] = sentiment

In [ ]:
df.head(1)

,item_id,date,title,category,content,link,photo_link,tokens,tokens_v2,entities,token_pos,top_key_freq,summary,sentiment
0,sportsv_20250319_1,2025-03-19,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,MLB,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...,"[《, 看看, MLB25-05, 》, 國聯, 雙雄, 的, 東京, 系列賽, 第二, 回...","[國聯, 雙雄, 東京, 系列賽, 回合, 佐佐木朗希, 大聯盟, 百英里, 速球, 方面,...","[(第二, ORDINAL, (22, 24)), (一, CARDINAL, (36, 3...","[(《, PARENTHESISCATEGORY), (看看, VE), (MLB25-05...","[(道奇, 4), (威力, 4), (東京, 3), (佐佐木朗希, 3), (速球, 3...",暫無,暫無


In [ ]:
df.columns

Index(['item_id', 'date', 'title', 'category', 'content', 'link', 'photo_link',
       'tokens', 'tokens_v2', 'entities', 'token_pos', 'top_key_freq',
       'summary', 'sentiment'],
      dtype='object')

In [ ]:
# This operation is the same as slicing
df=df[['item_id', 'date', 'category', 'title', 'content','sentiment', 'summary', 'top_key_freq', 'tokens',
       'tokens_v2', 'entities', 'token_pos', 'link', 'photo_link'
       ]]

In [ ]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,sportsv_20250319_1,2025-03-19,MLB,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,暫無,暫無,"[(道奇, 4), (威力, 4), (東京, 3), (佐佐木朗希, 3), (速球, 3...","[《, 看看, MLB25-05, 》, 國聯, 雙雄, 的, 東京, 系列賽, 第二, 回...","[國聯, 雙雄, 東京, 系列賽, 回合, 佐佐木朗希, 大聯盟, 百英里, 速球, 方面,...","[(第二, ORDINAL, (22, 24)), (一, CARDINAL, (36, 3...","[(《, PARENTHESISCATEGORY), (看看, VE), (MLB25-05...",https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...


In [ ]:
df.to_csv('sportsv_preprocessed.csv', sep='|', index=False)

In [ ]:
df = pd.read_csv('sportsv_preprocessed.csv', sep='|')

In [ ]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,sportsv_20250319_1,2025-03-19,MLB,【MLB25:東京系列賽】道奇強火援護佐佐木朗希，大谷翔平炸裂首轟助隊再斬小熊,《看看MLB25-05》國聯雙雄的東京系列賽第二回合，除了小熊企圖想扳回一城外，更是佐佐木朗...,暫無,暫無,"[('道奇', 4), ('威力', 4), ('東京', 3), ('佐佐木朗希', 3)...","['《', '看看', 'MLB25-05', '》', '國聯', '雙雄', '的', ...","['國聯', '雙雄', '東京', '系列賽', '回合', '佐佐木朗希', '大聯盟'...","[NerToken(word='第二', ner='ORDINAL', idx=(22, 2...","[('《', 'PARENTHESISCATEGORY'), ('看看', 'VE'), (...",https://www.sportsv.net/articles/118239,https://bucket-image.inkmaginecms.com/version/...
